In [1]:
import Pkg; 
cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`


In [3]:
pwd()

"/Users/ivanspirandelli/Doktor/Code/MorphoMol/MorphoMolNotebooks"

In [11]:
folder = "../../Data/collected_simulation_results/rwm_ma_2_6r7m/"
mol_type = "6r7m"
#folder = "../../Data/collected_simulation_results/2su/6r7m/"

id_index = 1
separator = "."
max = 1
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        max = max < parse(Int, split(file, separator)[id_index]) ? parse(Int, split(file, separator)[id_index]) : max
    end
end

min_Es = [Inf for _ in 1:max]
evaluation_strings = ["" for _ in 1:max]
for file in readdir(folder)
    if split(file, ".")[end] == "jld2"
        @load "$folder$file" input output
        id = parse(Int, split(file, separator)[id_index])
        exp_template_centers = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["template_centers"]
        exp_state = MorphoMol.Utilities.TWOTMVSU_EXPERIMENTAL_ASSEMBLY[mol_type]["state"]
        theta = MorphoMol.Utilities.average_offset_distance(exp_template_centers, input["template_centers"], exp_state, output["states"][argmin(output["Es"])])
        string = "$(id): $(input["T"]) | $(output["αs"][end]) | $(output["Es"][1]), $(minimum(output["Es"])), $(output["Es"][end]) | $(theta)"
        evaluation_strings[id] = string
        min_Es[id] = minimum(output["Es"])
    end
end

In [12]:
argmin(min_Es)

52

In [13]:
for elem in evaluation_strings
    println(elem)
end

1: 2.0 | 0.009289576 | 8207.8099251459, 8157.643098217686, 8159.273390004637 | 3.4340120173798776
2: 2.0 | 0.014425842 | 8216.50130661928, 8169.778638136508, 8170.842245211545 | 25.148082431829284
3: 2.0 | 0.05547243 | 8219.578303949487, 8164.394661938923, 8164.394661938923 | 18.77767347268998
4: 2.0 | 0.07270128 | 8219.578304032446, 8168.690511135349, 8172.267786846769 | 3.467947565255353
5: 2.0 | 0.038006186 | 8219.578304166915, 8159.165367396533, 8159.165367396533 | 1.7353177937694986
6: 2.0 | 0.0015514767 | 8953.820367168366, 8160.509905862564, 8166.0434743075875 | 6.175240131359553
7: 2.0 | 0.027309671 | 8219.57830463194, 8164.3629962719815, 8166.722035929966 | 5.746739735981404
8: 2.0 | 0.08325099 | 8219.57830428532, 8173.258655483985, 8186.277673662464 | 22.246868014435414
9: 2.0 | 0.030520245 | 8219.57830449305, 8159.488216494082, 8175.047465069298 | 19.663972785286404
10: 2.0 | 0.0058394275 | 8496.547017250798, 8166.936597287754, 8186.590770897971 | 6.538346887302918
11: 2.0 |

In [11]:
for file in readdir(folder)
    try
        @load "$folder$file" input output
        mindex = argmin(output["Es"])
        state = output["states"][mindex]
        id = parse(Int, split(file, "_")[id_index])
        radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
        MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(folder)$(id)", input["n_mol"], length(input["template_radii"]))
    catch e
        println(e)
        continue
    end
end

# Print Single Assembly Path

In [38]:
@load "../../Data/collected_simulation_results/2su/6r7m/10_6r7m.jld2" input output


2-element Vector{Symbol}:
 :input
 :output

In [39]:
input["template_centers"]

3×1206 Matrix{Float64}:
 -27.8796   -26.6674   -26.5597   …  -31.3903   -31.5191   -32.6934
   9.063      9.54921    9.03875       8.50467    8.25724    9.091
  -4.79258   -4.1493    -2.71995      -1.62113   -0.2168    -2.14403

In [41]:
mindex = argmin(output["Es"])
lower = 500
upper = 100
out_folder = "tmp/assembly_path/"
try
    rm(out_folder, recursive=true)
catch 
end
mkdir(out_folder)
radii = collect(Base.Iterators.flatten([input["template_radii"] for _ in 1:input["n_mol"]]))
for (i,j) in enumerate(maximum([mindex-lower,1]):minimum([mindex+upper, length(output["Es"])]))
    state = output["states"][j]
    MorphoMol.Utilities.state_to_poly(MorphoMol.Utilities.get_flat_realization(state, input["template_centers"]), radii, "$(out_folder)$(i)", input["n_mol"], length(input["template_radii"]))
end


# Dispersed State Energies

In [19]:
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    println("$(split(file, "_")[1]) ", 2*E)
end

1ei7 8255.3356279267
2tmv 8130.426867538925
6r7m 8219.578304239454
6sae 8185.217724310747
6sag 8174.157936120334
